In [1]:

import numpy as np
import pandas as pd
import re
import pickle

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential, load_model
from keras.layers import Dense, Embedding, LSTM, Bidirectional,Flatten,Dropout
from keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.utils.np_utils import to_categorical

In [4]:

data = pd.read_csv('/content/Ugly_Words_Cleaned.csv')

In [7]:
data.head()

,Unnamed: 0,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,...,473,474,475,476,477,478,479,480,481,482,483,484,485,486,487,488,489,490,491,492,493,494,495,496,497,498,499,500,501,502,503,504,505,506,507,508,509,510,511,512
0,0,-0.006094,0.065966,-0.046845,0.040648,-0.030130,-0.023790,0.056527,-0.065852,-0.047428,-0.056160,0.088488,-0.051742,0.019496,-0.002006,0.027364,-0.009147,-0.091029,-0.032635,0.002336,-0.032071,-0.042377,0.007383,0.015644,-0.042522,0.042829,0.007519,-0.027325,-0.045158,-0.021668,0.008290,0.023466,0.052940,-0.030344,0.029409,0.027079,0.006934,-0.064930,0.011496,-0.059324,...,0.024493,0.012440,-0.038833,0.036442,-0.030081,0.074626,0.045845,-0.047664,0.033448,-0.026130,-0.041222,0.014812,-0.041096,-0.025044,0.071706,-0.027785,0.043153,0.017411,-0.034988,0.010471,-0.005788,-0.003608,0.022213,0.076433,-0.079104,0.025912,-0.072889,0.075390,-0.028353,-0.046562,-0.001789,-0.012119,-0.087569,0.070395,0.041583,-0.024782,-0.018403,0.064336,0.023899,0
1,1,0.006361,0.025959,-0.027952,0.001518,-0.013413,-0.067714,-0.001843,0.091285,0.008502,-0.001199,0.046238,-0.084894,0.009045,-0.019875,0.016725,-0.056786,-0.089404,0.003482,-0.074631,0.092147,-0.011003,0.044720,-0.011596,0.014175,-0.025227,-0.032197,0.054663,-0.015005,0.017968,0.051361,0.005318,0.017381,0.048456,0.030366,-0.010908,0.023627,-0.025335,0.014164,-0.072274,...,-0.017617,0.051566,0.025877,-0.019085,0.018181,0.014180,-0.051931,0.072933,0.051571,0.008811,0.021542,0.056723,-0.036580,0.017011,0.053777,0.023837,0.065040,-0.011565,-0.046644,-0.080186,0.006143,-0.012163,0.057589,-0.021593,0.038647,-0.060047,-0.075744,0.004248,0.085780,0.009040,0.065440,0.043355,-0.042679,0.044132,-0.003048,-0.039173,0.022263,0.015416,-0.004290,0
2,2,0.048829,-0.060494,-0.011249,0.067714,0.031886,0.018584,0.017908,0.029567,0.043314,0.025356,0.039855,-0.068241,-0.033927,0.001750,-0.016082,-0.087752,-0.074289,-0.016032,-0.013859,0.041890,-0.025151,-0.050797,0.081614,0.005414,-0.008674,-0.042772,-0.050961,0.043938,-0.031682,-0.019450,-0.022859,0.022562,-0.049356,-0.063293,-0.060750,-0.034437,-0.011536,-0.042644,0.069053,...,0.023761,-0.024235,-0.072609,-0.047378,-0.011907,0.080831,0.004519,-0.062234,-0.001170,0.024669,-0.070842,0.017096,-0.033856,-0.030742,-0.042252,0.037881,0.074398,0.001571,0.006344,-0.010115,-0.025161,0.079277,0.038562,0.001469,-0.042112,-0.078997,-0.035002,-0.036753,-0.032487,-0.032209,0.060470,0.032786,-0.083513,0.050025,0.042526,-0.000471,-0.040855,0.000244,0.061038,0
3,3,-0.024284,0.035660,0.040260,-0.041432,0.037575,-0.044228,-0.043456,0.025993,0.045415,-0.009352,0.015814,-0.061261,-0.034509,0.009614,-0.087867,-0.042819,0.028624,0.004667,-0.060060,-0.016125,-0.071600,0.021748,-0.001089,0.054235,-0.058266,-0.080125,0.040128,0.052721,-0.024309,-0.080134,-0.084016,-0.010754,0.032563,0.011730,-0.062315,0.025751,-0.059691,-0.052057,-0.005240,...,0.022025,-0.044064,-0.047238,0.068293,-0.068364,0.073577,-0.074130,0.046873,0.059490,-0.012571,0.001885,-0.025272,0.004599,-0.019556,-0.009868,-0.087938,-0.009424,0.018898,0.047014,-0.029821,0.036958,-0.008463,-0.080246,0.016471,0.012518,0.050836,-0.047858,-0.057605,-0.041624,-0.039929,0.027021,0.059985,0.044810,0.021452,-0.053405,-0.074764,-0.003455,-0.055596,-0.027016,1
4,4,-0.065497,-0.028049,-0.026457,-0.027984,0.030934,-0.066535,-0.035914,-0.036650,0.044182,0.033349,0.107539,-0.048549,-0.021603,0.029933,0.038250,-0.072755,0.038196,0.021220,0.004549,-0.009129,-0.051661,0.009047,0.004510,0.008522,-0.074400,0.049719,0.028056,0.020028,-0.019425,0.046790,0.049753,0.014766,0.058855,0.001822,0.060028,0.064304,0.014638,-0.023256,-0.058496,...,-0.022632,-0.031790,0.080133,-0.084210,-0.058235,-0.000373,-0.037745,0.036185,0.016646,-0.000792,0.035372,0.050890,0.012551,-0.049534,0.032592,-0.051424,0.020990,0.007733,-0.015857,-0.054653,-0.009364,-0.011444,-0.005667,0.080311,0.017774,-0.015277,-0.008410,0.055033,-0.045128,-0.064190,0.019930,0.014888,-0.019798,0.014917,-0.029

In [18]:
data = data.drop(['Unnamed: 0'], axis=1)

In [19]:
data.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,473,474,475,476,477,478,479,480,481,482,483,484,485,486,487,488,489,490,491,492,493,494,495,496,497,498,499,500,501,502,503,504,505,506,507,508,509,510,511,512
0,-0.006094,0.065966,-0.046845,0.040648,-0.030130,-0.023790,0.056527,-0.065852,-0.047428,-0.056160,0.088488,-0.051742,0.019496,-0.002006,0.027364,-0.009147,-0.091029,-0.032635,0.002336,-0.032071,-0.042377,0.007383,0.015644,-0.042522,0.042829,0.007519,-0.027325,-0.045158,-0.021668,0.008290,0.023466,0.052940,-0.030344,0.029409,0.027079,0.006934,-0.064930,0.011496,-0.059324,-0.020344,...,0.024493,0.012440,-0.038833,0.036442,-0.030081,0.074626,0.045845,-0.047664,0.033448,-0.026130,-0.041222,0.014812,-0.041096,-0.025044,0.071706,-0.027785,0.043153,0.017411,-0.034988,0.010471,-0.005788,-0.003608,0.022213,0.076433,-0.079104,0.025912,-0.072889,0.075390,-0.028353,-0.046562,-0.001789,-0.012119,-0.087569,0.070395,0.041583,-0.024782,-0.018403,0.064336,0.023899,0
1,0.006361,0.025959,-0.027952,0.001518,-0.013413,-0.067714,-0.001843,0.091285,0.008502,-0.001199,0.046238,-0.084894,0.009045,-0.019875,0.016725,-0.056786,-0.089404,0.003482,-0.074631,0.092147,-0.011003,0.044720,-0.011596,0.014175,-0.025227,-0.032197,0.054663,-0.015005,0.017968,0.051361,0.005318,0.017381,0.048456,0.030366,-0.010908,0.023627,-0.025335,0.014164,-0.072274,-0.015295,...,-0.017617,0.051566,0.025877,-0.019085,0.018181,0.014180,-0.051931,0.072933,0.051571,0.008811,0.021542,0.056723,-0.036580,0.017011,0.053777,0.023837,0.065040,-0.011565,-0.046644,-0.080186,0.006143,-0.012163,0.057589,-0.021593,0.038647,-0.060047,-0.075744,0.004248,0.085780,0.009040,0.065440,0.043355,-0.042679,0.044132,-0.003048,-0.039173,0.022263,0.015416,-0.004290,0
2,0.048829,-0.060494,-0.011249,0.067714,0.031886,0.018584,0.017908,0.029567,0.043314,0.025356,0.039855,-0.068241,-0.033927,0.001750,-0.016082,-0.087752,-0.074289,-0.016032,-0.013859,0.041890,-0.025151,-0.050797,0.081614,0.005414,-0.008674,-0.042772,-0.050961,0.043938,-0.031682,-0.019450,-0.022859,0.022562,-0.049356,-0.063293,-0.060750,-0.034437,-0.011536,-0.042644,0.069053,-0.008054,...,0.023761,-0.024235,-0.072609,-0.047378,-0.011907,0.080831,0.004519,-0.062234,-0.001170,0.024669,-0.070842,0.017096,-0.033856,-0.030742,-0.042252,0.037881,0.074398,0.001571,0.006344,-0.010115,-0.025161,0.079277,0.038562,0.001469,-0.042112,-0.078997,-0.035002,-0.036753,-0.032487,-0.032209,0.060470,0.032786,-0.083513,0.050025,0.042526,-0.000471,-0.040855,0.000244,0.061038,0
3,-0.024284,0.035660,0.040260,-0.041432,0.037575,-0.044228,-0.043456,0.025993,0.045415,-0.009352,0.015814,-0.061261,-0.034509,0.009614,-0.087867,-0.042819,0.028624,0.004667,-0.060060,-0.016125,-0.071600,0.021748,-0.001089,0.054235,-0.058266,-0.080125,0.040128,0.052721,-0.024309,-0.080134,-0.084016,-0.010754,0.032563,0.011730,-0.062315,0.025751,-0.059691,-0.052057,-0.005240,-0.004868,...,0.022025,-0.044064,-0.047238,0.068293,-0.068364,0.073577,-0.074130,0.046873,0.059490,-0.012571,0.001885,-0.025272,0.004599,-0.019556,-0.009868,-0.087938,-0.009424,0.018898,0.047014,-0.029821,0.036958,-0.008463,-0.080246,0.016471,0.012518,0.050836,-0.047858,-0.057605,-0.041624,-0.039929,0.027021,0.059985,0.044810,0.021452,-0.053405,-0.074764,-0.003455,-0.055596,-0.027016,1
4,-0.065497,-0.028049,-0.026457,-0.027984,0.030934,-0.066535,-0.035914,-0.036650,0.044182,0.033349,0.107539,-0.048549,-0.021603,0.029933,0.038250,-0.072755,0.038196,0.021220,0.004549,-0.009129,-0.051661,0.009047,0.004510,0.008522,-0.074400,0.049719,0.028056,0.020028,-0.019425,0.046790,0.049753,0.014766,0.058855,0.001822,0.060028,0.064304,0.014638,-0.023256,-0.058496,0.038138,...,-0.022632,-0.031790,0.080133,-0.084210,-0.058235,-0.000373,-0.037745,0.036185,0.016646,-0.000792,0.035372,0.050890,0.012551,-0.049534,0.032592,-0.051424,0.020990,0.007733,-0.015857,-0.054653,-0.009364,-0.011444,-0.005667,0.080311,0.017774,-0.015277,-0.008410,0.055033,-0.045128,-0.064190,0.019930,0.0

In [20]:
X = data.loc[ : , data.columns != '512'] 

In [21]:
Y = data.loc[ : , data.columns == '512'] 

In [22]:

X_train, X_test, Y_train, Y_test = train_test_split(data, Y, test_size=0.3, random_state = 24)

In [25]:

def prepare_targets(y_train, y_test):
    le = LabelEncoder()
    le.fit(y_train)
    le.fit(y_test)
    y_train_enc = le.transform(y_train)
    y_test_enc = le.transform(y_test)
    
    return y_train_enc, y_test_enc

In [26]:
y_train,y_test = prepare_targets(Y_train,Y_test)

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_label.py:235: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_label.py:268: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [30]:
X_train.shape

(1304, 513)

In [32]:
Y_train.shape

(1304, 1)

In [33]:
Y_test.shape

(559, 1)

In [34]:
X_test.shape

(559, 513)

In [38]:
leng = 3

In [61]:

model = Sequential()
model.add(Embedding(leng, 50, input_length=500))
model.add(Bidirectional(LSTM(128)))
model.add(Dropout(0.5))
model.add(Dense(1,activation='sigmoid'))
model.add(Dense(1,activation='relu'))
model.compile(loss='mse', optimizer='adam',metrics=['accuracy'])
model.summary()


Model: "sequential_14"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_8 (Embedding)      (None, 500, 50)           150       
_________________________________________________________________
bidirectional_9 (Bidirection (None, 256)               183296    
_________________________________________________________________
dropout_9 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_10 (Dense)             (None, 1)                 257       
Total params: 183,703
Trainable params: 183,703
Non-trainable params: 0
_________________________________________________________________


In [62]:
history=model.fit(X_train, y_train, batch_size=128, epochs=5, validation_data=[X_test, y_test])

Epoch 1/5
11/11 [==============================] - 29s 2s/step - loss: 0.2466 - accuracy: 0.5486 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 2/5
11/11 [==============================] - 26s 2s/step - loss: 0.2390 - accuracy: 0.5596 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 3/5
11/11 [==============================] - 26s 2s/step - loss: 0.2216 - accuracy: 0.5737 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 4/5
11/11 [==============================] - 25s 2s/step - loss: 0.1888 - accuracy: 0.7027 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 5/5
11/11 [==============================] - 26s 2s/step - loss: 0.1173 - accuracy: 0.9557 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
